In [ ]:
# 必要なライブラリのimport
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
# 複数列をまとめる関数を定義
def create_nested_dict(df, prefix):
    emotions = ["Joy", "Sadness", "Anticipation", "Surprise", "Anger", "Fear", "Disgust", "Trust"]
    # 該当する列をフィルタリング
    cols = [col for col in df.columns if col.startswith(prefix)]
    if not cols:
        return pd.Series([None] * len(df))

    # 辞書を作成
    return df[cols].apply(lambda row: {emotion.lower(): row[f"{prefix}{emotion}"] for emotion in emotions if f"{prefix}{emotion}" in row.index}, axis=1)

In [ ]:
# 今回利用するデータセットのダウンロード
dataset_df = pd.read_csv("https://github.com/ids-cv/wrime/raw/master/wrime-ver1.tsv", sep="\t")

# データの調整
# 1. Sentence列のデータから改行を削除
dataset_df["Sentence"] = dataset_df["Sentence"].str.replace("\\n", "")

# 2. SentenceとUserIDの列名を変更
dataset_df = dataset_df.rename(columns={"Sentence": "sentence", "UserID": "user_id", "Datetime": "datetime"})

# 3. 複数の列をまとめた新しい列を作成
dataset_df["writer"] = create_nested_dict(dataset_df, "Writer_")
dataset_df["reader1"] = create_nested_dict(dataset_df, "Reader1_")
dataset_df["reader2"] = create_nested_dict(dataset_df, "Reader2_")
dataset_df["reader3"] = create_nested_dict(dataset_df, "Reader3_")
dataset_df["avg_readers"] = create_nested_dict(dataset_df, "Avg. Readers_")

# 4. 訓練データ／検証データ／テストデータに分割し、Train/Dev/Test列を削除
train_df = dataset_df[dataset_df["Train/Dev/Test"] == "train"].reset_index(drop=True)[["sentence", "user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"]]
dev_df = dataset_df[dataset_df["Train/Dev/Test"] == "dev"].reset_index(drop=True)[["sentence", "user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"]]
test_df = dataset_df[dataset_df["Train/Dev/Test"] == "test"].reset_index(drop=True)[["sentence", "user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"]]

# 5. CPUでの実行時間を考慮し、訓練データ数を調整 (例: 5000件)
# もしメモリや時間に余裕があれば、より多くのデータを使用することも可能
num_train_samples = 5000
train_df = train_df[:num_train_samples]

train_df.head()

,sentence,user_id,datetime,writer,reader1,reader2,reader3,avg_readers
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1,2012/07/31 23:48,"{'joy': 0, 'sadness': 1, 'anticipation': 2, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's..."
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,1,2012/08/02 23:09,"{'joy': 3, 'sadness': 0, 'anticipation': 3, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's..."
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1,2012/08/05 00:50,"{'joy': 1, 'sadness': 1, 'anticipation': 1, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's..."
3,眠い、眠れない。,1,2012/08/08 01:36,"{'joy': 0, 'sadness': 2, 'anticipation': 1, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's..."
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,1,2012/08/09 22:24,"{'joy': 2, 'sadness': 1, 'anticipation': 3, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's..."


In [ ]:
# trainデータの最初の'avg_readers'を確認
train_df.loc[0, "avg_readers"]

{'joy': np.int64(0),
 'sadness': np.int64(2),
 'anticipation': np.int64(0),
 'surprise': np.int64(0),
 'anger': np.int64(0),
 'fear': np.int64(0),
 'disgust': np.int64(0),
 'trust': np.int64(0)}

In [ ]:
# 感情のkeyをリストにする
em_keys = list(train_df.loc[0, "avg_readers"].keys())

In [ ]:
# それぞれのDataFrameに'emotion'列を追加し、数値が一番大きいavg_readersのkeyを格納する
train_df["emotion"] = [em_keys.index(max(em, key=em.get)) for em in train_df["avg_readers"]]

In [ ]:
# sentence列とemotion列のみ残す
train_df = train_df.drop(["user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"], axis=1)
train_df.head()

,sentence,emotion
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,3
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,3
3,眠い、眠れない。,1
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,0


In [ ]:
# emotionの内訳を確認
train_df.groupby("emotion").count()

,sentence
emotion,
0,1421
1,1100
2,1064
3,887
4,124
5,139
6,231
7,34


In [ ]:
# シンボリックリンクの警告を表示させない
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# sentenceの文章をトークナイザーで前処理する
# 軽量な多言語対応モデルのトークナイザーを指定
tokenizer_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

tokenized_data = tokenizer(
    train_df["sentence"].to_list(),
    return_tensors="np",  # TensorFlow形式のテンソルで返す
    padding=True,         # 短いシーケンスをパディング
    truncation=True,      # 長いシーケンスを切り捨て
    max_length=512        # モデルが扱える最大長に設定 (多くのモデルで一般的)
)
tokenized_data = dict(tokenized_data)  # 辞書形式に変換しておく
tokenized_data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

{'input_ids': array([[  101,  1960, 27849, ...,     0,     0,     0],
        [  101,  2187,  4348, ...,     0,     0,     0],
        [  101,  4352,  3425, ...,     0,     0,     0],
        ...,
        [  101,  5915,  4704, ...,     0,     0,     0],
        [  101,  2531,  2046, ...,     0,     0,     0],
        [  101,   124,  4388, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [ ]:
# emotion列をndarrayにする
emotions = np.array(train_df["emotion"])

In [ ]:
# モデルを作成して学習させる
model_name = "distilbert-base-multilingual-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8, from_pt=True)

# オプティマイザや学習率を設定
# 軽量モデルやCPU実行の場合、学習率やバッチサイズの調整が効果的なことがあります。
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) # 一般的な学習率
# model.compileのlossは、TFAutoModelForSequenceClassification.from_pretrainedでfrom_pt=TrueとしてPyTorchの重みをロードした場合や、
# カスタム損失関数を使わない場合は、通常モデル内部で処理されるか、フレームワーク標準の損失関数が使われます。
# Hugging FaceのTFモデルでは、loss引数をcompile時に指定せず、fit時に直接ラベルを渡すことで内部損失が使われることが多いです。
# ここでは明示的に`model.hf_compute_loss`を使っていますが、もしこれが原因で別のエラーが出る場合は、
# `loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)` のような標準的な損失関数を試すことも検討できます。
# ただし、Hugging Faceのドキュメントでは `model.hf_compute_loss` または `loss=model.hf_compute_loss()` の使用が推奨される場合があります。
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy']) # metricsを追加して精度を監視

# バッチサイズとエポック数を設定
# CPU実行ではバッチサイズを小さめ (例: 8, 16) に、エポック数も少なめ (例: 1-3) にすると時間短縮になります。
batch_size = 16
epochs = 1 # まずは1エポックで試すなど、調整してください

print("ファインチューニングを開始します。CPUでは時間がかかる場合があります...")
# tokenized_dataは辞書形式である必要があります。{'input_ids': ..., 'attention_mask': ...}
# emotionsはラベルのnumpy配列です。
model.fit(dict(tokenized_data), emotions, batch_size=batch_size, epochs=epochs)
print("ファインチューニングが完了しました。")

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

ファインチューニングを開始します。CPUでは時間がかかる場合があります...
313/313 [==============================] - 4073s 13s/step - loss: 1.6292 - accuracy: 0.3460
ファインチューニングが完了しました。


In [ ]:
# testデータについても同様に前処理をしていく
test_df.head()

,sentence,user_id,datetime,writer,reader1,reader2,reader3,avg_readers
0,汗めっちゃかいた(),49,2016/06/18 15:25,"{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's..."
1,1人だけ春みたいなかっこしててはずい,49,2016/06/18 15:40,"{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's..."
2,……はあ；；；；,49,2016/06/18 21:36,"{'joy': 2, 'sadness': 0, 'anticipation': 2, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's..."
3,あぁーテスト勉強(),49,2016/06/19 21:17,"{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's..."
4,ハシが狂っていく様が儚げで見てて辛かったけど、なんか好き…,49,2016/06/19 22:14,"{'joy': 1, 'sadness': 0, 'anticipation': 1, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 1, 'anticipation': 0, 's..."


In [ ]:
test_df["emotion"] = [em_keys.index(max(em, key=em.get)) for em in test_df["avg_readers"]]
test_df = test_df.drop(["user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"], axis=1)
test_df.head()

,sentence,emotion
0,汗めっちゃかいた(),0
1,1人だけ春みたいなかっこしててはずい,1
2,……はあ；；；；,0
3,あぁーテスト勉強(),1
4,ハシが狂っていく様が儚げで見てて辛かったけど、なんか好き…,0


In [ ]:
test_tokenized_data = tokenizer(
    test_df["sentence"].to_list(),
    return_tensors="np",
    padding=True,
    truncation=True,
    max_length=512
)
test_tokenized_data = dict(test_tokenized_data)
test_tokenized_data

{'input_ids': array([[  101,  4881,  1965, ...,     0,     0,     0],
        [  101,   122,  2179, ...,     0,     0,     0],
        [  101,   100,   100, ...,     0,     0,     0],
        ...,
        [  101,  2195,  2149, ...,     0,     0,     0],
        [  101,  4422,  1925, ...,     0,     0,     0],
        [  101,  2187, 10906, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [ ]:
test_emotions = np.array(test_df["emotion"])

In [ ]:
# testデータで予測を実施
print("テストデータで予測を開始します...")
predictions = model.predict(test_tokenized_data)
print("予測が完了しました。")

テストデータで予測を開始します...
63/63 [==============================] - 457s 7s/step
予測が完了しました。


In [ ]:
# 実行結果をそのまま表示してみる
predictions

TFSequenceClassifierOutput(loss=None, logits=array([[ 1.2336202 ,  1.8170155 ,  0.06503231, ..., -0.92531365,
        -0.41812363, -2.111428  ],
       [ 0.51502126,  1.3424896 ,  2.008079  , ..., -1.0890867 ,
        -0.4783417 , -1.9399431 ],
       [ 0.24313208,  1.1714299 ,  0.2947519 , ..., -0.67478883,
         0.01797448, -1.7969599 ],
       ...,
       [ 2.6979847 ,  0.87359154,  0.6555102 , ..., -1.3059564 ,
        -0.9481427 , -2.030782  ],
       [ 0.7238036 ,  1.7616888 ,  0.1271132 , ..., -0.7594707 ,
        -0.3336035 , -2.040851  ],
       [ 1.57909   ,  1.3994522 ,  1.1390417 , ..., -1.3623347 ,
        -0.59695476, -2.3682287 ]], dtype=float32), hidden_states=None, attentions=None)

In [ ]:
# 一番大きい確率のものを予測結果とする形で調整する
# model.predictの出力がTFSequenceClassifierOutputオブジェクトの場合、.logitsでアクセス
pred_emotions = np.argmax(predictions.logits, axis=1)
pred_emotions

array([1, 2, 1, ..., 0, 1, 0])

In [ ]:
# 実際の値を確認
test_emotions

array([0, 1, 0, ..., 2, 6, 2])

In [ ]:
from sklearn import metrics

# 混同行列
print("混同行列:")
print(metrics.confusion_matrix(test_emotions, pred_emotions))

混同行列:
[[322 130  92  50   0   0   0   0]
 [ 65 218  84  26   0   0   0   0]
 [145  87 254  27   0   0   0   0]
 [ 30  56  47  53   0   0   0   0]
 [  2   7   9   4   0   0   0   0]
 [ 15 101  54  16   0   0   0   0]
 [ 10  47  36   6   0   0   0   0]
 [  1   4   1   1   0   0   0   0]]


In [ ]:
# 正解率
accuracy = metrics.accuracy_score(test_emotions, pred_emotions)
print(f"正解率: {accuracy:.4f}")

正解率: 0.4235
